In [1]:
using Revise
using Pkg

ENV["PYTHON"] = Sys.which("python")
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")
Pkg.build("PyCall")
using FileIO
using JLD2
using RiskSensitiveSAC

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/8c86e48c0db1564a1d49548d3515ced5d604c408/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`


┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /home/kang/DRC_Project/DRC_Trajectron/Manifest.toml:0


┌ Info: Number of Julia Thread(s): 1
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:39
┌ Info: CUDA Device: NVIDIA GeForce RTX 3070 Laptop GPU
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:40


┌ Info: Python executable used by PyCall: /home/kang/anaconda3/envs/trajectron++/bin/python
└ @ RiskSensitiveSAC /home/kang/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:41


In [2]:
include("$(@__DIR__)/../scripts/default_params/params_data_gaussian.jl");

dtr = 0.4;                                                                          # replanning time interval [s]
tcalc = 0.2;                                                                        # pre-allocated control computation time [s] (< dtr)

model_dir = normpath(joinpath(@__DIR__, "../CrowdNav/crowd_nav/data/output_om_sarl_radius_0.4")) # directory of the trained policy
env_config = "env.config"                                                           # environment config file name
policy_config = "policy.config"                                                     # policy config file name
policy_name = "sarl"                                                                # policy name


include("$(@__DIR__)/../scripts/parameter_setup_crowd_nav.jl");

In [3]:
scene_loader, controller, w_init, ado_inputs, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param, cnt_param,
                 cost_param=cost_param,
                 dtc=dtc,
                 prediction_steps=prediction_steps,
                 ego_pos_init_vec=ego_pos_init_vec,
                 ego_pos_goal_vec=ego_pos_goal_vec,
                 target_speed=target_speed,
                 sim_horizon=sim_horizon,
                 verbose=true);

Scene Mode: data
CrowdNav Controller


Loaded evaluation data from /home/kang/DRC_Project/DRC_Trajectron/Trajectron-plus-plus/experiments/processed/eth_test.pkl
Looking at the eth_test.pkl sequence, data_id 0, start_idx 905


PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/kang/.julia/packages/PyCall/twYvK/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'NameError'>
NameError("name 'policy_factory' is not defined",)
  File "/home/kang/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 145, in configure_rl_robot
    if state == 'p'


In [ ]:
result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec, target_speed,
                  measurement_schedule, target_trajectory, pos_error_replan, 
                  ado_inputs_init=ado_inputs);

In [ ]:
display_log(result.log)

In [ ]:
result.total_cnt_cost

In [ ]:
result.total_pos_cost

In [ ]:
result.total_col_cost

In [ ]:
result.total_cnt_cost + result.total_pos_cost + result.total_col_cost

In [ ]:
minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                          for w in result.w_history])

In [ ]:
make_gif(result, dtplot=0.02, fps=50, xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5., filename="8_crowd_nav_data.gif")

In [ ]:
save("8_crowd_nav_data.jld2", "result", result)